In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# import the neccessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [5]:
# load the data and label in separate lists
words_directory = '/content/drive/MyDrive/Dataset/Static-Word'
data = []
label = []
i = 0
for word in os.listdir(words_directory):
    words_path = os.path.join(words_directory, word)
    for image in os.listdir(words_path):
        image_path = os.path.join(words_path, image)
        img = load_img(image_path, target_size=(64, 64))
        img = img_to_array(img)
        data.append(img)
        label.append(i)
    i+=1

# print("-----Completed-----")

In [6]:
# convert the lists to numpy arrays
data = np.array(data)
label = np.array(label)

In [7]:
# check the size of the data and labels
print(data.shape)
print(label.shape)

(37864, 64, 64, 3)
(37864,)


In [8]:
# converting the numeral labels to array like label
label = to_categorical(label)

In [9]:
# shuffle the data
n = np.arange(data.shape[0])
np.random.shuffle(n)
data_shuffled = data[n]
label_shuffled = label[n]

print("---Shuffling Completed Succesfully---")

---Shuffling Completed Succesfully---


In [10]:
data_shuffled = data_shuffled/255.0

In [11]:
# split the data into train and test data
# train-size: 80%
# val-size: 20%
x_train, x_test, y_train, y_test = train_test_split(data_shuffled, label_shuffled, test_size=0.1, stratify=label, random_state=12)

In [13]:
# make a sequential model

model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(37, activation='softmax'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 31, 31, 64)       256       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 31, 31, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 64)      

In [15]:
# compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [17]:
# fit the train data into the model
b = model.fit(
    x_train, y_train,
    epochs=40,
    callbacks=[callback],
)

Epoch 1/40
1065/1065 [==============================] - 30s 18ms/step - loss: 0.2285 - accuracy: 0.9303
Epoch 2/40
1065/1065 [==============================] - 19s 18ms/step - loss: 0.0433 - accuracy: 0.9874
Epoch 3/40
1065/1065 [==============================] - 19s 18ms/step - loss: 0.0375 - accuracy: 0.9900
Epoch 4/40
1065/1065 [==============================] - 19s 18ms/step - loss: 0.0090 - accuracy: 0.9976
Epoch 5/40
1065/1065 [==============================] - 19s 18ms/step - loss: 0.0346 - accuracy: 0.9908
Epoch 6/40
1065/1065 [==============================] - 19s 18ms/step - loss: 0.0181 - accuracy: 0.9952
Epoch 7/40
1065/1065 [==============================] - 19s 18ms/step - loss: 0.0131 - accuracy: 0.9972


In [18]:
len(b.history['loss'])

7

In [19]:
y_pred_custom = model.predict(x_test)

In [20]:
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_custom, axis=1))
acc*100

96.98970161077371

In [ ]:
# save the model
model.save('alphabet_model.h5')